<pre>영화 개봉 예정 흥행 예측후 db로 저장</pre>

- 네이버 개봉예정영화(72개) 클롤링

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

In [4]:
url = 'https://search.naver.com/search.naver?where=nexearch&sm=tab_etc&qvt=0&query=%EA%B0%9C%EB%B4%89%EC%98%88%EC%A0%95%EC%98%81%ED%99%94'
driver = webdriver.Chrome('APIKey/chromedriver')
driver.get(url)
time.sleep(2)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_44416\2770024186.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('APIKey/chromedriver')


- table은 영화개봉예정 page당 8개의 영화로 구성되어있다.

In [5]:
driver.find_element(By.CSS_SELECTOR, 'div.cm_paging_area > .pgs > .pg_next').click()

In [ ]:
table = driver.find_elements(By.CSS_SELECTOR, '.cm_content_wrap .card_item')

In [ ]:
page_html = BeautifulSoup(driver.page_source, 'html.parser')
table_html = page_html.select('.cm_content_wrap .card_item')

In [ ]:
table_url = []
table_url.append(table_html[0].select_one('.this_text')['href'])

In [ ]:
url = 'https://search.naver.com/search.naver' + table_url[0]
url

'https://search.naver.com/search.naver?where=nexearch&sm=tab_etc&pkid=68&os=26500642&qvt=0&query=%EA%B7%80%EB%A9%B8%EC%9D%98%20%EC%B9%BC%EB%82%A0%3A%20%EC%95%84%EC%82%AC%EC%BF%A0%EC%82%AC%20%ED%8E%B8'

In [ ]:
table[0].find_element(By.CSS_SELECTOR, '.this_text').click()
time.sleep(2)

- 영화를 각각 클릭후 기본정보, 감독/출연, 포토를 클릭하여 가져온다.

- 액션	모헌	판타지	SF	드라마	공포	스릴러	미스터리	로맨스	budget	charactor	director	vote_weight

In [ ]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [ ]:
# 전체 > 포스터
poster_path = soup.select_one('img._img')["src"]

In [ ]:
# 기본 정보: 1, 감독/출연: 2, 포토: 5
# 기본 정보: 1
driver.find_elements(By.CSS_SELECTOR, '.tab_list > li')[1].find_element(By.CSS_SELECTOR, 'a').click()

In [ ]:
driver.find_elements(By.CSS_SELECTOR, 'dl.info .info_group')[1].find_element(By.CSS_SELECTOR, 'dt').text

'장르'

In [ ]:
# 기본정보 > 장르
infos = driver.find_elements(By.CSS_SELECTOR, 'dl.info .info_group')
for info in infos:
    if(info.find_element(By.CSS_SELECTOR, 'dt').text == "장르"):
        genres_str = info.find_element(By.CSS_SELECTOR, 'dd').text
geres = genres_str.split(', ')

In [ ]:
# 기본정보 > 소개
introduction = driver.find_element(By.CSS_SELECTOR, 'p._content_text').text

In [ ]:
# 감독/출연: 2
driver.find_elements(By.CSS_SELECTOR, '.tab_list > li')[2].find_element(By.CSS_SELECTOR, 'a').click()

In [ ]:
# 감독/출연 > 감독: 0
director = driver.find_elements(By.CSS_SELECTOR, 'div.cast_box')[0].find_element(By.CSS_SELECTOR,'ul.cast_list span._text').text

In [ ]:
# 감독/출연 > 출연: 1
mainCharactor = driver.find_elements(By.CSS_SELECTOR, 'div.cast_box')[1].find_elements(By.CSS_SELECTOR,'ul.cast_list li')[0].find_element(By.CSS_SELECTOR, 'span._text').text

<pre>크롤링</pre>

In [ ]:
movie = []
for page in tqdm(range(9)):
    url = 'https://search.naver.com/search.naver?where=nexearch&sm=tab_etc&qvt=0&query=%EA%B0%9C%EB%B4%89%EC%98%88%EC%A0%95%EC%98%81%ED%99%94'
    driver = webdriver.Chrome('APIKey/chromedriver')
    driver.get(url)
    time.sleep(2)
    for click in range(page):
        driver.find_element(By.CSS_SELECTOR, 'div.cm_paging_area > .pgs > .pg_next').click()
        time.sleep(1)
    page_html = BeautifulSoup(driver.page_source, 'html.parser')
    table_html = page_html.select('.cm_content_wrap .card_item')
    table_url = []
    for idx in range(8):
        try:
            table_url.append(table_html[idx].select_one('.this_text')['href'])
        except:
            pass
    driver.close()
    for idx in range(len(table_url)):
        url = 'https://search.naver.com/search.naver' + table_url[idx]
        driver = webdriver.Chrome('APIKey/chromedriver')
        driver.get(url)
        time.sleep(2)
        # 포스터 url Path
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        try:
            poster_path = soup.select_one('img._img')["src"]
        except:
            poster_path = ""
        driver.find_elements(By.CSS_SELECTOR, '.tab_list > li')[1].find_element(By.CSS_SELECTOR, 'a').click()
        # 기본정보 > 장르
        infos = driver.find_elements(By.CSS_SELECTOR, 'dl.info .info_group')
        for info in infos:
            if(info.find_element(By.CSS_SELECTOR, 'dt').text == "장르"):
                genres_str = info.find_element(By.CSS_SELECTOR, 'dd').text
        geres = genres_str.split(', ')
        # 기본정보 > 소개
        try:
            introduction = driver.find_element(By.CSS_SELECTOR, 'p._content_text').text
        except:
            introduction = ""
        # 감독/출연: 2
        driver.find_elements(By.CSS_SELECTOR, '.tab_list > li')[2].find_element(By.CSS_SELECTOR, 'a').click()
        # 감독/출연 > 감독: 0
        try:
            director = driver.find_elements(By.CSS_SELECTOR, 'div.cast_box')[0].find_element(By.CSS_SELECTOR,'ul.cast_list span._text').text
        except:
            director = ""
        # 감독/출연 > 출연: 1
        try:
            mainCharactor = driver.find_elements(By.CSS_SELECTOR, 'div.cast_box')[1].find_elements(By.CSS_SELECTOR,'ul.cast_list li')[0].find_element(By.CSS_SELECTOR, 'span._text').text
        except:
            mainCharactor = ""
        movie.append([poster_path, geres, director, mainCharactor, introduction])
        driver.close()

  0%|          | 0/9 [00:00<?, ?it/s]

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_33928\3446743936.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('APIKey/chromedriver')
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_33928\3446743936.py:21: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('APIKey/chromedriver')


In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(movie, columns=['poster_path', 'genres', 'director', 'mainCharactor', 'introduction'])

In [ ]:
# null 값 확인
df.drop(df[df.poster_path == ""].index, inplace=True)
df.drop(df[df.genres == ""].index, inplace=True)
df.drop(df[df.director == ""].index, inplace=True)
df.drop(df[df.mainCharactor == ""].index, inplace=True)
df.drop(df[df.introduction == ""].index, inplace=True)

In [ ]:
df.to_csv('data/movie_data_predict_noscreen.csv', index=False)

In [7]:
import pandas as pd
df = pd.read_csv('data/movie_data_predict_noscreen.csv')

In [8]:
movie_data_name = pd.read_csv('data/movie_data_Name.csv')

In [9]:
movie_data_id = pd.read_csv('data/movie_data_ID.csv')

In [10]:
movie_data_name.head(1)

,title,budget,genres,country,charactor,director,vote_score,vote_count,vote_weight
0,토르: 러브 앤 썬더,250000000,"['액션', '모험', '판타지']",['United States of America'],Christian Bale,Taika Waititi,6.778,1802,10.000812


In [11]:
movie_data_id.head(1)

,title,budget,genres,country,charactor,director,vote_score,vote_count,vote_weight
0,616037,250000000,"[28, 12, 14]",[0],3894,55934,6.778,1802,10.000812


In [12]:
# charactor id 중 3550 ~ 3620(70)까지는 없다.
for i in range(5000):
    if(len(movie_data_id[movie_data_id.charactor == i].index) > 0):
        print(i, end=" ")

2 3 4 5 13 14 20 31 32 33 35 48 49 50 52 62 63 64 65 66 75 85 98 100 103 109 110 112 113 114 116 118 131 133 134 138 139 140 141 147 155 177 190 192 193 199 200 204 205 207 209 212 216 227 228 246 258 259 270 287 289 290 326 327 328 335 336 349 350 351 368 378 380 382 400 417 418 448 449 453 454 478 480 500 501 504 514 515 516 517 518 519 520 522 524 529 530 531 569 585 587 588 589 617 643 650 658 677 679 689 690 707 723 724 738 741 776 777 780 819 820 821 825 827 854 855 862 879 880 882 883 884 887 921 934 935 938 955 974 976 980 1003 1004 1018 1019 1037 1038 1062 1100 1101 1115 1121 1125 1137 1146 1158 1159 1160 1161 1162 1164 1204 1205 1229 1230 1231 1233 1234 1241 1244 1245 1246 1247 1248 1269 1271 1276 1281 1283 1285 1327 1328 1331 1333 1336 1337 1338 1339 1341 1370 1372 1459 1461 1518 1524 1532 1533 1548 1562 1581 1602 1607 1619 1620 1622 1639 1640 1650 1666 1679 1733 1736 1748 1749 1771 1772 1796 1810 1811 1812 1813 1892 1893 1896 1903 1904 1920 1922 1923 1925 1932 1933 1934 193

In [13]:
# 감독 id중 3150 ~ 3220(70)까지는 없다.
for i in range(5000):
    if(len(movie_data_id[movie_data_id.director == i].index) > 0):
        print(i, end=" ")

1 7 8 24 31 39 40 42 43 59 68 108 138 150 172 185 190 197 201 223 224 240 280 309 323 357 366 380 455 468 488 508 510 518 525 559 564 578 607 608 628 638 648 664 686 707 716 769 793 814 819 834 865 876 879 880 893 898 956 957 974 1011 1012 1032 1035 1038 1044 1060 1071 1090 1098 1103 1108 1114 1150 1152 1176 1188 1201 1215 1223 1224 1243 1255 1269 1270 1333 1414 1461 1496 1524 1577 1614 1650 1704 1707 1723 1724 1726 1744 1748 1749 1769 1776 1777 1788 1884 1899 1921 1927 1978 2000 2034 2036 2042 2053 2087 2127 2128 2150 2163 2171 2176 2178 2199 2209 2223 2226 2228 2239 2291 2293 2294 2303 2352 2357 2388 2406 2419 2460 2461 2523 2589 2636 2662 2675 2679 2690 2710 2746 2765 2778 2836 2873 2878 2880 2917 2945 2991 3026 3027 3028 3045 3047 3055 3146 3224 3227 3239 3288 3289 3290 3310 3317 3388 3415 3417 3451 3460 3556 3632 3893 3950 3964 3974 4014 4027 4037 4065 4081 4109 4135 4163 4275 4385 4387 4415 4429 4453 4489 4499 4586 4587 4590 4600 4610 4723 4746 4755 4759 4762 4782 4786 4818 4918 

In [14]:
# 기존에 같은 연출자는 없다.
for char in df.mainCharactor:
    if(len(movie_data_name[movie_data_name.charactor == char].index) > 0):
        print(char, end="  ")

In [15]:
# 기존에 같은 감독자는 없다.
for dir in df.director:
    if(len(movie_data_name[movie_data_name.charactor == dir].index) > 0):
        print(dir, end="  ")

In [16]:
# 같은 애니매이션이 감독과 연출이 같다
df[df.mainCharactor.duplicated() == True]

,poster_path,genres,director,mainCharactor,introduction
25,https://search.pstatic.net/common?type=o&size=...,['애니메이션'],소토자키 하루오,하나에 나츠키,꺽쇠 까마귀가 일러준 다음 임무지는 남남동. 임무로 향하는 도중 탄지로는 최종 선별...
36,https://search.pstatic.net/common?type=o&size=...,['애니메이션'],소토자키 하루오,하나에 나츠키,"‘탄지로’와 ‘네즈코’, ‘젠이츠’, ‘이노스케’는 귀살대원들의 실종사건을 조사하기..."
40,https://search.pstatic.net/common?type=o&size=...,['애니메이션'],소토자키 하루오,하나에 나츠키,‘탄지로’와 ‘네즈코’는 귀살대 중에서도 최고의 계급인 지주들 앞으로 끌려간다. 탄...


In [17]:
# charactor id 중 3550 ~ 3620(70)까지는 없다.
df['mainCharactorID'] = 0
charactor = {}
i = 3550
for char in df.mainCharactor:
    try:
        charactor[char]
    except:
        charactor[char] = i
    i += 1
for idx in df.index:
    df.mainCharactorID[idx] = charactor[df.mainCharactor[idx]]

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_44416\3733998328.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.mainCharactorID[idx] = charactor[df.mainCharactor[idx]]


In [18]:
# 감독 id중 3150 ~ 3220(70)까지는 없다.
df['directorID'] = 0
direc = {}
i = 3150
for dir in df.director:
    try:
        direc[dir]
    except:
        direc[dir] = i
    i += 1
for idx in df.index:
    df.directorID[idx] = direc[df.director[idx]]

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_44416\498819551.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.directorID[idx] = direc[df.director[idx]]


- 공연실황 : 11, 22/ 뮤지컬 : 30, 38 -> 지우기 || 멜로/로맨스 : 1, 7, 45, 48 -> 로맨스

In [19]:
df.drop(df[df.genres.apply(lambda x: '공연실황' in x)].index, inplace=True)
df.drop(df[df.genres.apply(lambda x: '뮤지컬' in x)].index, inplace=True)

In [20]:
df[df.genres.apply(lambda x: '멜로/로맨스' in x)].genres

1     ['멜로/로맨스', '드라마', '코미디']
7                   ['멜로/로맨스']
45                  ['멜로/로맨스']
48           ['멜로/로맨스', '드라마']
Name: genres, dtype: object

In [21]:
import ast
df.genres = df.genres.apply(lambda x: ast.literal_eval(x))

In [22]:
genres_list = ["액션", "모험" ,"판타지", "가족", "애니메이션", "코미디", "SF", "드라마", "공포", "스릴러", "범죄", "미스터리", "음악", "역사", "TV 영화", "전쟁", "멜로/로맨스", "다큐멘터리", "서부"]
df_genre_each = pd.DataFrame(columns=genres_list)
for idx in df.index:
    genres = df.genres[idx]
    # 한 줄 0으로 초기화
    df_genre_each.loc[idx] = 0
    for genre in genres:
        df_genre_each.loc[idx][str(genre)] = 1  # 한 줄 값 부여

In [23]:
df_genre_each.rename(columns= {'멜로/로맨스':'로맨스'},inplace=True)

In [24]:
dataframe = pd.concat([df, df_genre_each], axis=1)

In [25]:
dataframe.to_csv('data/movie_data_predict_noscreenML.csv', index=False)